<a href="https://colab.research.google.com/github/laragazzadelsole/AMD-Exam/blob/main/Amazon_US_costumer_Review_Link_Analysis_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project description

The task is to implement a ranking system based on the PageRank index using the «Amazon US Customer Review» dataset, published on Kaggle under the amazon.com conditions of use. The entities to be ranked can be either customers or products. In the first case, there will be a link between two customers if they have reviewed at least a same product, while in the second one two products will be linked if they have been reviewed at least by a same customer. Note that this dataset is composed by several CSV files, each related to a specific product category: you are free to choose how much such files have to be processed.


 <font size="5">**1. Initial Setup**</font>

**1.1 Data Import**

In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = "saragironi"
os.environ['KAGGLE_KEY'] = "4b28e3c84038475619b3fff13d413869"
!pip install kaggle --upgrade
!kaggle datasets download -d cynthiarempel/amazon-us-customer-reviews-dataset --unzip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
100% 21.0G/21.0G [03:37<00:00, 112MB/s]
100% 21.0G/21.0G [03:37<00:00, 103MB/s]


**1.2 Initializing Spark**

In [ ]:
#installing Java 8

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!java -version

openjdk version "11.0.18" 2023-01-17
OpenJDK Runtime Environment (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1)
OpenJDK 64-Bit Server VM (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1, mixed mode, sharing)


In [ ]:
# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

# unzip the spark file to the current folder
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

In [ ]:
# install findspark and pyspark using pip

!pip install -q findspark
!pip install pyspark


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 19.9 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=613d0c13412ed19a0b8c9d2c03516a4c474ac2a3572dd1716a123b373655bd0f
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark


In [ ]:
#import findspark 

import findspark
findspark.init()

In [ ]:
#create a SparkSession

from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

spark

<font size="5">**2. The Data**</font> \

I have decided to focus the link analysis on the customers that left reviews on electronical products. The reason for this choice is that this kind of reviews tend to be more technical and less subjective due to the nature of the product, compared to other categories (such as Books, Music, Grocery, etc). 
Therefore, there could be a higher number of "helpful_votes", which is an important element for my purpose. The chosen categories are: Electronics and PC. These two categories are correlated as people purchasing a pc on Amazon could also probably buy a mouse or other electronical items. 

In [ ]:
# The dataset is divided per categories. Here I define only those in which I am interested in. 

df_el = spark.read.csv('amazon_reviews_us_Electronics_v1_00.tsv', sep='\t', header=True)
df_pc = spark.read.csv('amazon_reviews_us_PC_v1_00.tsv', sep='\t', header=True)
df_el.show(10)
df_pc.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41409413|R2MTG1GCZLR2DK|B00428R89M|     112201306|yoomall 5M Antenn...|     Electronics|          5|            0|          0|   N|                Y|          Five Stars|       As described.| 2015-08-31|
|         US|   49668221|R2HBOEM8LE9928|B000068O48|     734576678|Hosa GPM-103 3.5m...|     Electronics|          5|    

In [ ]:
df = df_pc.union(df_el)
type(df)

pyspark.sql.dataframe.DataFrame

In [ ]:
#Drop the columns that are not really useful for the purpose of this project

df = df.drop("marketplace","vine", "product_parent", "verfied_purchase", "review_headline", "review_body")
df.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_date: string (nullable = true)



In [ ]:
#The dataframe contains more than 10 millions rows. Here I want to compare the time difference between the action .count() and spark.sql commands.
# .count() will be used further as it's faster

%%time 
tot_rows = df.count()
print(f"Total number of rows: {tot_rows}")

Total number of rows: 10002423
CPU times: user 304 ms, sys: 26.7 ms, total: 331 ms
Wall time: 46.1 s


In [ ]:
%%time
df.createOrReplaceTempView("df_view")
total_rows = spark.sql("""SELECT COUNT(review_id) AS total_rows 
                        FROM df_view""")
total_rows.show()

+----------+
|total_rows|
+----------+
|  10002423|
+----------+

CPU times: user 374 ms, sys: 50.5 ms, total: 424 ms
Wall time: 1min 4s


As Spark is lazy the action count() is faster. For this reason I'll use this method instead of spark.sql() in the rest of the code

**2.1 Data Pre-processing**

In [ ]:
# Check if there are null values 

import pyspark.sql.functions as F

df.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in df.columns]).show()

+-----------+---------+----------+-------------+----------------+-----------+-------------+-----------+-----------------+-----------+
|customer_id|review_id|product_id|product_title|product_category|star_rating|helpful_votes|total_votes|verified_purchase|review_date|
+-----------+---------+----------+-------------+----------------+-----------+-------------+-----------+-----------------+-----------+
|          0|        0|         0|            0|              11|         11|           11|         11|               11|        333|
+-----------+---------+----------+-------------+----------------+-----------+-------------+-----------+-----------------+-----------+



In [ ]:
#Rows with missing data are deleted

df_complete = df.na.drop()
df_complete.count()

10002090

In [ ]:
# drop duplicates of reviews if any

df_final = df_complete.dropDuplicates(["review_id"])
type(df_final)

pyspark.sql.dataframe.DataFrame

In [ ]:
#check for anomalies in the values like star_rating > 5
#first cast the data type of "star_rating" from string to int

from pyspark.sql.types import IntegerType,BooleanType,DateType
from pyspark.sql.functions import col,when,count

df_final = df_final.withColumn("star_rating",df_final.star_rating.cast('int'))

df_final.select('review_id').where(df_final.star_rating>5).count()

0

In [ ]:
#check for consistencies: if "helpful_votes" is always lower or equal to "total_votes"

df_final = df_final.withColumn("helpful_votes",df_final.helpful_votes.cast('int'))
df_final = df_final.withColumn("total_votes",df_final.total_votes.cast('int'))

df_final.select('review_id').where(df_final.helpful_votes>df_final.total_votes).count()

0

The dataframe is now complete and there aren't inconsistencies. However, if I were to implement the algorithm on the dataset as a whole it would be too computationally intense. For this reason a subsample of 50000 rows is used.

In [ ]:
sub_df = df_final.limit(50000)

<font size="5">**3. Exploratory Data Analysis**</font> \

In [ ]:
#calculate the average number of reviews per person

from pyspark.sql.functions import count, desc, avg

# Group the data by person and count the number of reviews per person
reviews_per_person = sub_df.groupby("customer_id").count().select("count")

# Calculate the average number of reviews per person
avg_reviews_per_person = reviews_per_person.agg({"count": "avg"}).collect()[0][0]

print("The average number of reviews per person is:", avg_reviews_per_person)

The average number of reviews per person is: 1.0109792344865236


In [ ]:
#The average is very low so I check the first 10 customers by number of reviews.
# 6 is the maximum number of reviews left by a user.  

reviews_per_person.sort(desc("count")).show(10)

+-----+
|count|
+-----+
|    6|
|    5|
|    4|
|    4|
|    4|
|    4|
|    3|
|    3|
|    3|
|    3|
+-----+
only showing top 10 rows



In [ ]:
#The first 10 most reviewed items

sub_df.groupby('product_id', 'product_title').count().sort(desc("count")).show(10)

+----------+--------------------+-----+
|product_id|       product_title|count|
+----------+--------------------+-----+
|B0051VVOB2|Kindle Fire (Prev...|  111|
|B0083PWAPW|Kindle Fire HD 7"...|   85|
|B00JG8GOWU|Kindle Paperwhite...|   80|
|B006GWO5WK|Amazon Kindle 9W ...|   77|
|B00BWYQ9YE|Kindle Fire HDX 7...|   74|
|B003L1ZYYM|AmazonBasics High...|   68|
|B002Y27P3M|Kindle Keyboard, ...|   63|
|B0015T963C|Kindle Wireless R...|   61|
|B004XC6GJ0|ARRIS SURFboard D...|   57|
|B00I15SB16|Kindle, 6" Glare-...|   56|
+----------+--------------------+-----+
only showing top 10 rows



In [ ]:
# First 10 most useful reviews.

sub_df.select("product_title", "star_rating", "helpful_votes", "customer_id").sort(desc("helpful_votes")).show(10)

+--------------------+-----------+-------------+-----------+
|       product_title|star_rating|helpful_votes|customer_id|
+--------------------+-----------+-------------+-----------+
|Zune HD Video MP3...|          5|         2431|   52855449|
|SanDisk Extreme P...|          4|         1920|   51960937|
|Patagonia Kindle ...|          4|         1277|   12372811|
|Samsung Chromeboo...|          5|         1109|   41866357|
|Motorola SURFboar...|          4|          800|   28204599|
|Samsung Galaxy Ta...|          4|          637|   45035381|
|Datamancer The So...|          4|          618|   27055887|
|Samsung Galaxy Ta...|          5|          515|   15208771|
|Photive Hydra Wir...|          5|          437|   45843410|
|ASUS MeMOPad HD 7...|          1|          424|   53090839|
+--------------------+-----------+-------------+-----------+
only showing top 10 rows



In [ ]:
#First 10 days by total number of reviews

# first cast the data type of "review_date" from string to date format

from pyspark.sql.functions import to_date, lit
sub_df = sub_df.withColumn("review_date",col("review_date").cast(DateType()))

In [ ]:
#The highest numbers of reviews are left in January of years 2014 and 2015. 
#The possible explanation is that Christmas is the moment where most products are purchased
#and after few days or weeks trying the product he/she leaves the reviews on Amazon. 

sub_df.groupby('review_date').count().sort(desc("count")).show(10)

+-----------+-----+
|review_date|count|
+-----------+-----+
| 2015-01-04|  125|
| 2015-01-05|  105|
| 2015-01-07|   99|
| 2015-01-03|   97|
| 2015-06-03|   91|
| 2015-01-13|   91|
| 2015-01-06|   88|
| 2014-12-31|   84|
| 2015-08-18|   82|
| 2015-01-09|   82|
+-----------+-----+
only showing top 10 rows



<font size="5">**4. PageRank Algorithm**</font> \


Link analysis purpose: pagerank of users + helpful_votes and the dummy variable "verified_purchase" as a weight. People with higher index are more reliable while users with lower link analyisis and no helpful_votes might be bots that try to inflate their product. 
To conclude we can cluster users. Best users are those who have reviewed many objects and have done helpful reviews. Their reviews should be displayed in the top. 

In order to create the graph I want to compare two methods. The first one relies on the udf() a suer defined function that allows to apply a function directly to a Pyspark dataframe. It is very expensive, however, it is better than for loops used in the second 

**4.1 Creation of the Graph - Method with udf()**

In [ ]:
#The first step is to create the edges of the graph. 
#An edge exists when two customers have reviewed the same product
from pyspark.sql.functions import collect_set, sort_array, udf, explode
from pyspark.sql.types import ArrayType, StringType, StructType, StructField

# group by product_id and use collect_set() to aggregate the customer_id into an ArrayType to create a list of the customers by product
custom_by_prod = sub_df.groupBy('product_id').agg(collect_set('customer_id').alias('customer_id'))
custom_by_prod.show(10)

+----------+--------------------+
|product_id|         customer_id|
+----------+--------------------+
|016642966X|          [19955871]|
|0511189877|          [33355906]|
|0972683275|[53074039, 435844...|
|1394860919|          [16522736]|
|1400501466|[51932300, 48648451]|
|1400501776|          [49886899]|
|1400532620|          [51280446]|
|1400532655|          [17086455]|
|140053271X|[25652547, 195277...|
|1400599997|          [19325576]|
+----------+--------------------+
only showing top 10 rows



In [ ]:
#METHOD WITH UDF 
%%time

# The function create_edges() creates an edge between two customers who reviewed the same product
# by creating a tuple containing the customer ID of the two people. 
def create_edges(customer_id):
    edges = []
    for i in range(len(customer_id)):
        for j in range(i+1, len(customer_id)):
            edge = tuple(sorted([customer_id[i], customer_id[j]]))
            edges.append(edge)
    return edges

CPU times: user 6 µs, sys: 1 µs, total: 7 µs
Wall time: 11.2 µs


In [ ]:
# edge_udf() allows to apply the create_edges() function to a PySpark DataFrame. 
# the UDF will return an array of tuples, each of which has two string fields.
# However udf() is very expensive

%%time

edge_udf = udf(create_edges, ArrayType(StructType([
    StructField('src', StringType()),
    StructField('dst', StringType())])))

# edges is a new DataFrame with a row for each edge between customers that rated the same product ID

df_edges = custom_by_prod.select('customer_id').withColumn('edges', edge_udf(sort_array('customer_id'))).select(explode('edges').alias('edge'))
#df_edges.show(10)


CPU times: user 19.8 ms, sys: 1.69 ms, total: 21.5 ms
Wall time: 409 ms


In [ ]:
#JUST TO TEST 

#In order to verify that the edges have been created correctly I create a dictionary from custom_by_prod 
#and display the first 5 elements. I choose customer_id '53074039' because it is the first one 
# with edges and I retrieve the same customer_id from edges dataframe. If the edges in common coincide the 
#dataframe edges has been created correctly 

# convert the DataFrame to a list of Row objects and create a dictionary
custom_dict = {row['product_id']: row['customer_id'] for row in custom_by_prod.collect()}

# print the first 10 entries in the dictionary
print(dict(list(custom_dict.items())[:5]))

{'016642966X': ['19955871'], '0511189877': ['33355906'], '0972683275': ['53074039', '43584487', '29997920', '41490022', '26324308', '26780671', '13630979', '11988204'], '1394860919': ['16522736'], '1400501466': ['51932300', '48648451']}


In [ ]:
#The edges linked to customer 53074039 coincide both in the dataframe and in the dictionary. 
# Therefore we can go on with the analysis. 

df_edges.filter(df_edges.edge.dst == '53074039' ).show()

+--------------------+
|                edge|
+--------------------+
|[11988204, 53074039]|
|[13630979, 53074039]|
|[26324308, 53074039]|
|[26780671, 53074039]|
|[29997920, 53074039]|
|[41490022, 53074039]|
|[43584487, 53074039]|
+--------------------+



In [ ]:
#create the network 
%%time

#create lists from customer_id and edges to iterate on them 

custom_list = list(sub_df.select('customer_id').toPandas()['customer_id'])

edges_list = list(df_edges.select('edge').toPandas()['edge'])
#print(edges_list[:10])

CPU times: user 3.25 s, sys: 292 ms, total: 3.54 s
Wall time: 3min 27s


In [ ]:
#now let's substitute the nodes with some indexes to save memory and use them later in the PageRank implementation 
# Get list of unique nodes
n_int_list = list(set([e[0] for e in edges_list] + [e[1] for e in edges_list]))
n_int_list[:5]

['29898246', '26292187', '48612261', '11147560', '19739573']

In [ ]:
nodes = range(len(n_int_list))
links = range(len(edges_list))
nodes, links

(range(0, 26968), range(0, 118183))

In [ ]:
# Create a dictionary mapping nodes to a corresponding index

%%time
n_dict = {node: index for index, node in enumerate(n_int_list)}

# Replace node names with their indices in the edge list
links_idx_list = [(n_dict[a], n_dict[b]) for (a, b) in edges_list]
print(links_idx_list)

[(19256, 19075), (19256, 14371), (19256, 13951), (19256, 24660), (19256, 850), (19256, 12065), (19256, 20752), (19075, 14371), (19075, 13951), (19075, 24660), (19075, 850), (19075, 12065), (19075, 20752), (14371, 13951), (14371, 24660), (14371, 850), (14371, 12065), (14371, 20752), (13951, 24660), (13951, 850), (13951, 12065), (13951, 20752), (24660, 850), (24660, 12065), (24660, 20752), (850, 12065), (850, 20752), (12065, 20752), (18658, 6626), (11073, 20327), (11073, 3934), (11073, 23617), (11073, 11002), (20327, 3934), (20327, 23617), (20327, 11002), (3934, 23617), (3934, 11002), (23617, 11002), (3204, 18368), (3204, 10111), (18368, 10111), (21394, 3504), (23852, 4713), (23852, 4895), (23852, 16234), (4713, 4895), (4713, 16234), (4895, 16234), (24366, 4548), (16011, 3459), (21577, 16026), (26124, 19723), (10055, 17836), (10055, 12514), (17836, 12514), (5356, 9041), (25567, 7878), (7234, 21440), (7234, 985), (21440, 985), (20522, 24609), (656, 26294), (17057, 20257), (22624, 22527), 

In [ ]:
#create a dictionaty with a numeric label for each node and each edge
node_id = {c: i for i, c in enumerate(n_int_list)}
links_id = {c: i for i, c in enumerate(edges_list)}
print(node_id)

{'29898246': 0, '26292187': 1, '48612261': 2, '11147560': 3, '19739573': 4, '20416708': 5, '17894336': 6, '11437496': 7, '3024679': 8, '44088955': 9, '34982870': 10, '1780229': 11, '12883134': 12, '43156467': 13, '50076193': 14, '28552669': 15, '43969902': 16, '36417055': 17, '29471306': 18, '40381544': 19, '35334468': 20, '15988880': 21, '52617185': 22, '40452438': 23, '19227673': 24, '2978768': 25, '15937434': 26, '19461283': 27, '11491763': 28, '20633641': 29, '19558728': 30, '36639768': 31, '18905668': 32, '25071327': 33, '9305168': 34, '36206281': 35, '8475514': 36, '14300531': 37, '25284383': 38, '51518186': 39, '48169886': 40, '48558662': 41, '18774246': 42, '52495779': 43, '12287498': 44, '36043218': 45, '41037180': 46, '42592468': 47, '46840310': 48, '34125718': 49, '11360889': 50, '44948499': 51, '13331217': 52, '14854126': 53, '25413849': 54, '43530142': 55, '2772692': 56, '38208054': 57, '38465628': 58, '13166892': 59, '50110564': 60, '45947575': 61, '25516746': 62, '328475

In [ ]:
import networkx as nx

G = nx.Graph()
 
for n in nodes:
    node = G.add_node(n)
 
for (a,b) in links_id:
    G.add_edge(node_id[a], node_id[b])

print('Number of nodes:', G.number_of_nodes())
print('Number of edges:', G.number_of_edges())
print('Is the graph connected?', nx.is_connected(G))

Number of nodes: 26968
Number of edges: 118183
Is the graph connected? False


**4.2 Creation of the Graph - Method with for loops**

In [ ]:
#METHOD WITH FOR LOOPS 
%%time
from pyspark.sql.functions import collect_set
from itertools import combinations

# group by product_id and collect the customer_ids into a set
#custom_by_prod = sub_df.groupBy('product_id').agg(collect_set('customer_id').alias('customer_id'))

# create an empty list to store the edges
edges = []

# iterate through each row of custom_by_prod dataframe
for row in custom_by_prod.collect():
    customer_ids = row.customer_id
    # generate all possible combinations of the customer_ids for each product
    customer_pairs = combinations(customer_ids, 2)
    # append each customer pair as an edge to the edges list
    for pair in customer_pairs:
        edges.append(pair)

# convert the edges list to a PySpark dataframe
edges_df = spark.createDataFrame(edges, ['src', 'dst'])
#edges_df.show(10)

CPU times: user 3.36 s, sys: 19.7 ms, total: 3.38 s
Wall time: 4.09 s


In [ ]:
# edges_df.collect() returns a list of Row objects, where each Row represents a row in the pyspark dataframe.
# Then the for loop iterates on each Row and returns a list of tuples

%%time
ed_list = [tuple(row) for row in edges_df.collect()]
print(ed_list[:5])

[('53074039', '43584487'), ('53074039', '29997920'), ('53074039', '41490022'), ('53074039', '26324308'), ('53074039', '26780671')]
CPU times: user 746 ms, sys: 14.5 ms, total: 760 ms
Wall time: 1.75 s


In [ ]:
#now let's substitute the nodes with some indexes to save memory and use them later in the PageRank implementation 
# Get list of unique nodes
nodes_list = list(set([e[0] for e in ed_list] + [e[1] for e in ed_list]))
nodes_list[:5]

['26292187', '29898246', '48612261', '11147560', '19739573']

In [ ]:
# Create a dictionary mapping nodes to a corresponding index

%%time
node_dict = {node: index for index, node in enumerate(nodes_list)}

# Replace node names with their indices in the edge list
ed_idx_list = [(node_dict[a], node_dict[b]) for (a, b) in ed_list]
#print(ed_idx_list)

CPU times: user 46.6 ms, sys: 0 ns, total: 46.6 ms
Wall time: 46.9 ms


In [ ]:
%%time

import networkx as nx
graph = nx.Graph()

num_of_nodes = range(len(nodes_list))
 
for n in num_of_nodes:
    node = graph.add_node(n)
 
for (a,b) in ed_idx_list:
    graph.add_edge(num_of_nodes[a], num_of_nodes[b])

print('Number of nodes:', graph.number_of_nodes())
print('Number of edges:', graph.number_of_edges())
print('Is the graph connected?', nx.is_connected(graph))


Number of nodes: 26968
Number of edges: 118183
Is the graph connected? False
CPU times: user 258 ms, sys: 32.1 ms, total: 290 ms
Wall time: 292 ms


On average each node has around 4.38 edges.

The udf() is a lost faster than for loops. If I were to compare the two methods on the entire dataset the cost of using for loops would be definitely higher.  

In [ ]:
#The top 3 node with the most links  

node_degrees = list(graph.degree())
node_degrees.sort(key=lambda x: x[1], reverse=True)
top_nodes = [node_degrees[i][0] for i in range(3)]
print(dict(graph.degree(nbunch = top_nodes)).items())

dict_items([(6146, 160), (2802, 119), (5541, 113)])


**4.2 Creation of the Transition Matrix**

In [ ]:
#cast the type from string to int

ed_int_idx_list = [(int(x), int(y)) for (x, y) in ed_idx_list]
print(ed_int_idx_list[:5])

[(20752, 12065), (20752, 24660), (20752, 850), (20752, 14371), (20752, 13951)]


In [ ]:
# Create an empty dictionary with nodes as keys and an empty list as values
adjacency = {}

# The set() function is used to create a set of all unique nodes in the graph. 
# The adjacency dictionary is created by adding all the source and destination nodes of each edge to the set.

for n in set([e[0] for e in ed_int_idx_list] + [e[1] for e in ed_int_idx_list]):
    adjacency[n] = []
print(adjacency)


{0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: [], 10: [], 11: [], 12: [], 13: [], 14: [], 15: [], 16: [], 17: [], 18: [], 19: [], 20: [], 21: [], 22: [], 23: [], 24: [], 25: [], 26: [], 27: [], 28: [], 29: [], 30: [], 31: [], 32: [], 33: [], 34: [], 35: [], 36: [], 37: [], 38: [], 39: [], 40: [], 41: [], 42: [], 43: [], 44: [], 45: [], 46: [], 47: [], 48: [], 49: [], 50: [], 51: [], 52: [], 53: [], 54: [], 55: [], 56: [], 57: [], 58: [], 59: [], 60: [], 61: [], 62: [], 63: [], 64: [], 65: [], 66: [], 67: [], 68: [], 69: [], 70: [], 71: [], 72: [], 73: [], 74: [], 75: [], 76: [], 77: [], 78: [], 79: [], 80: [], 81: [], 82: [], 83: [], 84: [], 85: [], 86: [], 87: [], 88: [], 89: [], 90: [], 91: [], 92: [], 93: [], 94: [], 95: [], 96: [], 97: [], 98: [], 99: [], 100: [], 101: [], 102: [], 103: [], 104: [], 105: [], 106: [], 107: [], 108: [], 109: [], 110: [], 111: [], 112: [], 113: [], 114: [], 115: [], 116: [], 117: [], 118: [], 119: [], 120: [], 121: [], 122: [], 12

In [ ]:
# Iterate over the edges and add the target node to the adjacency list of the source node

for (n1, n2) in ed_int_idx_list:
    adjacency[n1].append(n2)
    adjacency[n2].append(n1)  # as the graph is undirected add the reverse edge as well

# Print the adjacency dictionary
print(dict(list(adjacency.items())[0:5]))

{0: [3801], 1: [6101], 2: [1332, 8963, 11192, 4538, 17797, 6150, 11610, 17523, 14134, 21680, 10783, 365, 24811, 3212, 8699, 22200, 21038, 14000, 785, 11621, 19079, 15870, 17619, 13524], 3: [1633, 3631, 26193, 3919, 10803, 24483], 4: [11810, 26469, 22649, 18672, 15118, 24618, 10152, 1202, 22676]}


In [ ]:
#an empty list is created to store the transition probabilities.

transition_matrix = []
for n1 in adjacency:
    for n2 in adjacency[n1]:
        # For each pair (n1,n2) of adjacent nodes, a tuple with 3 values is appended.
        #The first value n2 is the destination node, the second the source node, 
        #and the third value is the probability of moving from node n1 to node n2
        transition_matrix.append((n2, n1, 1./len(adjacency[n1])))
transition_matrix[:10]

[(3801, 0, 1.0),
 (6101, 1, 1.0),
 (1332, 2, 0.041666666666666664),
 (8963, 2, 0.041666666666666664),
 (11192, 2, 0.041666666666666664),
 (4538, 2, 0.041666666666666664),
 (17797, 2, 0.041666666666666664),
 (6150, 2, 0.041666666666666664),
 (11610, 2, 0.041666666666666664),
 (17523, 2, 0.041666666666666664)]

**4.3 Parallelization**

In [ ]:
# Caching the result of the transformation is one of the optimization tricks to
# improve the performance of the long-running PySpark applications/jobs.

import findspark
findspark.init()
from pyspark.sql import SparkSession

sparkContext=spark.sparkContext
edges_rdd = sparkContext.parallelize(transition_matrix).cache()
edges_rdd.sortByKey().take(10)

#the result is a list of triples of the form (i, j, m_ij)

[(0, 3801, 1.0),
 (1, 6101, 1.0),
 (2, 365, 0.041666666666666664),
 (2, 785, 0.041666666666666664),
 (2, 1332, 0.041666666666666664),
 (2, 3212, 0.041666666666666664),
 (2, 4538, 0.041666666666666664),
 (2, 6150, 0.041666666666666664),
 (2, 8699, 0.041666666666666664),
 (2, 8963, 0.041666666666666664)]

In [ ]:
# The initial vector represents the probability distribution of a random surfer starting in 
#any of the customer nodes. 

import numpy as np
n = len(num_of_nodes)
page_rank = np.ones(n)/n

# Set convergence threshold
threshold = 0.0001
max_rep = 500
# Initialize iteration count and residual error
iteration = 0
residual_error = float('inf')

# Run iterations until convergence
while residual_error > threshold and iteration < max_rep:
    # Calculate new page rank values 
    new_page_rank_values = edges_rdd.map(lambda i_j_mij: (i_j_mij[0], i_j_mij[2]*page_rank[i_j_mij[1]]))
                     
    new_page_rank_values = new_page_rank_values.reduceByKey(lambda a, b: a+b).collect()

    # Calculate residual error
    residual_error = sum(abs(c - page_rank[i]) for (i, c) in new_page_rank_values)

    page_rank = np.array([c for (i, c) in new_page_rank_values])

    #increment iteration count
    iteration += 1

    # Print final results
print("Converged after", iteration, "iterations")
print(page_rank[:30])

Converged after 500 iterations
[3.70809849e-05 3.70809849e-05 3.70456918e-05 3.70399491e-05
 7.37996402e-05 3.69946626e-05 3.70637910e-05 3.70534327e-05
 3.70519153e-05 3.70521985e-05 3.71613668e-05 3.70067550e-05
 3.70367652e-05 3.69945179e-05 3.69397823e-05 3.67153174e-05
 3.70662541e-05 3.71513740e-05 7.37902760e-05 3.70353990e-05
 3.69787105e-05 3.70453930e-05 3.67585659e-05 3.70275175e-05
 3.71192000e-05 3.70499537e-05 3.65843894e-05 3.69061164e-05
 3.66865256e-05 3.66290335e-05]


**4.4 Teleport Variation**

The implementation of PageRank in this way it is effective only if the graph is strongly connected and doesn't present dead ends nor spider traps. 
In the graph obtained dead ends are not possible because each edge is bidirectional, however it is not connected. This means that there could be spider traps. Spider traps are a set of nodes with no dead ends, but no arcs
out. To avoid this problem the calculation of PageRank is modified by allowing each random surfer a small probability beta to be teleported to a random page. 

In [ ]:
edges_rdd = sparkContext.parallelize(transition_matrix).cache()

n = len(num_of_nodes)
page_rank_t = np.ones(n)/n

# Set convergence threshold
threshold = 0.0001
max_rep = 500
# Initialize iteration count and residual error
iteration = 0
residual_error = float('inf')

# The conventional value of beta is 0.85
beta = 0.85

# Run iterations until convergence
while residual_error > threshold and iteration < max_rep:
    # Calculate new page rank values 
    new_page_rank_values_t = edges_rdd.map(lambda i_j_mij: (i_j_mij[0], i_j_mij[2]*page_rank_t[i_j_mij[1]]))
                     
    new_page_rank_values_t = new_page_rank_values_t.reduceByKey(lambda a, b: a+b).collect()

    # Calculate residual error
    residual_error = sum(abs(c - page_rank_t[i]) for (i, c) in new_page_rank_values_t)

    page_rank_t = np.array([beta * c + (1 - beta) * 1.0/n for (i, c) in new_page_rank_values_t])

    #increment iteration count
    iteration += 1

    # Print final results
print("Converged after", iteration, "iterations")
print(page_rank_t[:30])

Converged after 500 iterations
[3.70809849e-05 3.70809849e-05 3.70008380e-05 3.69910282e-05
 6.84402210e-05 3.69663803e-05 3.69987488e-05 3.69982633e-05
 3.69963460e-05 3.70058127e-05 3.70688592e-05 3.69774024e-05
 3.69905935e-05 3.69656761e-05 3.69471923e-05 3.68224508e-05
 3.70135105e-05 3.70544159e-05 6.83428105e-05 3.69878512e-05
 3.69573696e-05 3.69919890e-05 3.68478373e-05 3.69864593e-05
 3.70329815e-05 3.69930123e-05 3.68029542e-05 3.69799689e-05
 3.68141085e-05 3.68344842e-05]


In [ ]:
# Note the difference in the scores between PageRank scores and its variation 

diff_pagerank = page_rank - page_rank_t
diff_pagerank[:50]

array([ 0.00000000e+00,  0.00000000e+00,  4.48537926e-08,  4.89208648e-08,
        5.35941923e-06,  2.82822540e-08,  6.50422125e-08,  5.51694237e-08,
        5.55692752e-08,  4.63858129e-08,  9.25075708e-08,  2.93525821e-08,
        4.61717191e-08,  2.88418392e-08, -7.41002617e-09, -1.07133412e-07,
        5.27436151e-08,  9.69580413e-08,  5.44746549e-06,  4.75477880e-08,
        2.13409557e-08,  5.34040564e-08, -8.92713415e-08,  4.10582090e-08,
        8.62184731e-08,  5.69413437e-08, -2.18564760e-07, -7.38525736e-08,
       -1.27582884e-07, -2.05450669e-07, -1.96277425e-07, -2.53711168e-07,
        1.12299102e-06,  1.07674288e-06,  1.02984836e-06,  1.14455755e-06,
        1.10805004e-06,  1.15219674e-06,  9.55222099e-07,  1.11603729e-06,
        1.13739867e-06, -3.19179071e-07,  4.58598377e-08, -3.61827124e-07,
       -3.33167717e-07, -3.11480728e-07, -3.35498146e-07, -3.21441389e-07,
       -4.01039652e-07, -3.37273352e-07])

In [ ]:
# Create a dictionary to store the PageRank with teleport scores for each node
pagerank_scores = {}
for i, node in enumerate(node_dict.keys()):
    pagerank_scores[node] = page_rank[i]

# Print the top 10 nodes with the highest PageRank scores
top_nodes = sorted(pagerank_scores.items(), key=lambda x: x[1], reverse=True)[:20]
for node, score in top_nodes:
    print("Node {}: PageRank score = {:.5f}".format(node, score))

Node 38990896: PageRank score = 0.00015
Node 23531595: PageRank score = 0.00012
Node 44798805: PageRank score = 0.00011
Node 32081533: PageRank score = 0.00011
Node 30547826: PageRank score = 0.00011
Node 48507835: PageRank score = 0.00011
Node 13062763: PageRank score = 0.00009
Node 13362563: PageRank score = 0.00008
Node 13419426: PageRank score = 0.00008
Node 44048219: PageRank score = 0.00008
Node 30241212: PageRank score = 0.00008
Node 35036828: PageRank score = 0.00008
Node 41520709: PageRank score = 0.00008
Node 36658988: PageRank score = 0.00008
Node 42209651: PageRank score = 0.00008
Node 179522: PageRank score = 0.00008
Node 49647917: PageRank score = 0.00008
Node 29114490: PageRank score = 0.00008
Node 13690890: PageRank score = 0.00008
Node 35438801: PageRank score = 0.00008


In [ ]:
# Create a dictionary to store the PageRank with teleport scores for each node
pagerank_t_scores = {}
for i, node in enumerate(node_dict.keys()):
    pagerank_t_scores[node] = page_rank_t[i]

# Print the top 10 nodes with the highest PageRank scores
top_nodes_t = sorted(pagerank_t_scores.items(), key=lambda x: x[1], reverse=True)[:20]
for node, score in top_nodes_t:
    print("Node {}: PageRank score = {:.5f}".format(node, score))

Node 38990896: PageRank score = 0.00013
Node 23531595: PageRank score = 0.00010
Node 32081533: PageRank score = 0.00010
Node 44798805: PageRank score = 0.00010
Node 30547826: PageRank score = 0.00009
Node 48507835: PageRank score = 0.00009
Node 13062763: PageRank score = 0.00008
Node 13362563: PageRank score = 0.00007
Node 36658988: PageRank score = 0.00007
Node 42209651: PageRank score = 0.00007
Node 35036828: PageRank score = 0.00007
Node 13690890: PageRank score = 0.00007
Node 44048219: PageRank score = 0.00007
Node 44072084: PageRank score = 0.00007
Node 29114490: PageRank score = 0.00007
Node 44603630: PageRank score = 0.00007
Node 49647917: PageRank score = 0.00007
Node 47138494: PageRank score = 0.00007
Node 179522: PageRank score = 0.00007
Node 51695575: PageRank score = 0.00007


 <font size="5">5. Introduction of the "helpfulness vector"</font>

Pagerank values are all pretty low which is normal as in the exploratory analysis it was already shown that the most of the customers had few reviews in common. This is also because the sample chosen is very restricted compared to the size of the original dataset. 
Furthermore, many nodes have exactly the same score. For this reason I decide to introduce the variable "helpful_votes", to try to diversify between who left a useful review and who didn't, in order to create a better ordering. 

In [ ]:
# Group by customer_id and collect product_ids into a list
hv_df = sub_df.select("customer_id", "helpful_votes").groupBy("customer_id").sum("helpful_votes").alias("helpful_votes")
hv_df.show(10)

+-----------+------------------+
|customer_id|sum(helpful_votes)|
+-----------+------------------+
|   42706553|                 1|
|   19159455|                 0|
|   52808364|                 0|
|   12774562|                 6|
|   22295949|                 0|
|   47440766|                11|
|   12468782|                 0|
|   18658791|                 0|
|   22500595|                 0|
|   14340978|                 0|
+-----------+------------------+
only showing top 10 rows



In [ ]:
#find the total number of times the reviews have been considered useful 

filtered_hv_df = hv_df.filter(col("customer_id").isin(nodes_list))
total_hv = filtered_hv_df.groupBy().sum("sum(helpful_votes)").collect()[0][0]
total_hv

42957

In [ ]:
#calculate the helpfulness score for each customer  

filtered_hv_df = filtered_hv_df.withColumn('helpful_votes', col('sum(helpful_votes)') / total_hv)
filtered_hv_df.show(10)

+-----------+------------------+--------------------+
|customer_id|sum(helpful_votes)|       helpful_votes|
+-----------+------------------+--------------------+
|   52808364|                 0|                 0.0|
|   22295949|                 0|                 0.0|
|   47440766|                11|2.560700235118839...|
|   12468782|                 0|                 0.0|
|   14340978|                 0|                 0.0|
|   39528786|                 0|                 0.0|
|   23382296|                 1|2.327909304653490...|
|    7801327|                 0|                 0.0|
|   24225794|                 0|                 0.0|
|    2862763|                 0|                 0.0|
+-----------+------------------+--------------------+
only showing top 10 rows



In [ ]:
#dictinary with as key the customer_id and value the "helpfulness"
hv_dict = dict(filtered_hv_df.rdd.map(lambda x: (x[0], x[2])).collect())


In [ ]:
#pagerank scores

pagerank_t_scores = {}
for i, node in enumerate(node_dict.keys()):
    pagerank_t_scores[node] = page_rank_t[i]

In [ ]:
#sum the helpfulness score to the pagerank score for each customer 

final_score = {k: pagerank_t_scores.get(k, 0) + hv_dict.get(k, 0) for k in set(pagerank_t_scores) | set(hv_dict)}
final_score

{'26292187': 3.7080984870958175e-05,
 '29898246': 0.00013019735705709778,
 '48612261': 3.7000838032872567e-05,
 '11147560': 3.6991028201102104e-05,
 '19739573': 6.844022096476512e-05,
 '20416708': 3.696638032561225e-05,
 '17894336': 3.699874880675605e-05,
 '11437496': 8.355644940436204e-05,
 '3024679': 3.699634603740654e-05,
 '44088955': 3.700581270364808e-05,
 '34982870': 3.70688592040572e-05,
 '1780229': 3.6977402418099943e-05,
 '12883134': 3.6990593489993305e-05,
 '43156467': 3.6965676082032476e-05,
 '50076193': 3.6947192320021694e-05,
 '28552669': 6.0101543869069565e-05,
 '43969902': 0.000106850789641221,
 '36417055': 3.705441593881792e-05,
 '29471306': 0.00013818008962090782,
 '40381544': 3.698785120332943e-05,
 '35334468': 6.023646260622482e-05,
 '15988880': 3.699198896264543e-05,
 '52617185': 3.684783726139184e-05,
 '40452438': 3.6986459338496345e-05,
 '19227673': 3.703298152641949e-05,
 '2978768': 6.027210535849484e-05,
 '15937434': 3.680295420402555e-05,
 '19461283': 0.0003861

In [ ]:
# Print the top 10 nodes with the highest final score 

top_nodes_t = sorted(final_score.items(), key=lambda x: x[1], reverse=True)[:20]
for node, score in top_nodes_t:
    print("Node {}: Final score = {:.5f}".format(node, score))

Node 52855449: Final score = 0.05663
Node 12372811: Final score = 0.02976
Node 41866357: Final score = 0.02585
Node 45035381: Final score = 0.01487
Node 15208771: Final score = 0.01202
Node 53090839: Final score = 0.01128
Node 45843410: Final score = 0.01021
Node 51840028: Final score = 0.00953
Node 50695896: Final score = 0.00951
Node 18833993: Final score = 0.00921
Node 30669680: Final score = 0.00898
Node 35832624: Final score = 0.00865
Node 41766042: Final score = 0.00797
Node 31864628: Final score = 0.00779
Node 52860694: Final score = 0.00737
Node 32684861: Final score = 0.00709
Node 52810431: Final score = 0.00681
Node 51297632: Final score = 0.00600
Node 35360153: Final score = 0.00595
Node 51721371: Final score = 0.00567
